In [9]:
#!pip install sentence-transformers
# !pip install spacy-wordnet
# !pip install -U spacy --user

^C


In [1]:
import nltk
from nltk.corpus import wordnet as wn
import re
#!python -m spacy download en_core_web_sm
import spacy
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
print(spacy.__version__)

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
import pickle
from scipy import spatial
import os
print(os.getcwd())

3.2.0
C:\Users\yiehy\OneDrive\Desktop\cs425-nlc-project\6.Query Expansion


# Best performing flow (retrieve and re-rank)

In [2]:
bi_encoder_model = SentenceTransformer("../10.Fine-tuned Models/finetuned-bertbase-1epoch")
cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

C:\Users\yiehy\anaconda3\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


# On test set

In [17]:
##### UNCOMMENT AND RUN THE CODE BELOW IF YOU FACE THE ERROR "UNSUPPORTED PICKLE PROTOCOL: 5" WHEN RUNNING THIS CELL #####
# #!pip3 install pickle5
# import pickle5 as pickle
##########################################################################################################################

# Read embeddings
test_answer_embeddings = pickle.load(open("../4.Retrieval/finetuned_bertbase/finetuned_bertbase_test_answer_embeddings.pkl", 'rb'))

In [6]:
test_df = pd.read_csv("../0.Datasets/train_test_split/test.csv")
test_df

,Unnamed: 0,qid,docid,question,answer
0,13786,7817,31330,Can you have a positive return with a balance ...,Have you owned the stock for longer than 2015?...
1,11485,6304,105557,Oversimplify it for me: the correct order of i...,Great questions -- the fact that you're thinki...
2,12711,7115,43508,Definition of “U.S. source” for US non-residen...,The examples you provide in the question are c...
3,10336,5716,287327,Are car buying services worth it?,I have used car buying services through Costco...
4,15084,9016,580920,What makes a Company's Stock prices go up or d...,Here are some significant factors affect the c...
...,...,...,...,...,...
3410,6117,3567,579472,Will getting a new credit card and closing ano...,Several events will always result in a reducti...
3411,8218,4446,82194,Why are “random” deposits bad?,Random deposits are a bit like playing the lot...
3412,6545,3733,99619,Rental Application Fees,Slightly abbreviated version of the guidance f...
3413,1585,1417,303501,Market Cap lower than Shares Outstanding x Sha...,The definition of market cap is exactly shares...


In [7]:
question_answer_index_map = {}
for _,row in test_df.iterrows():
    if row["qid"] not in question_answer_index_map:
        question_answer_index_map[row["qid"]]= []
        question_answer_index_map[row["qid"]].append(row["docid"])
    else:
        question_answer_index_map[row["qid"]].append(row["docid"])

In [8]:
labels = [] # 2d list, with each value in the list being a list of docids that answer the same qid
for v in question_answer_index_map.values():
    labels.append(v)

In [9]:
question_map = {} # mapping of qid to question text
label_map = {} # mapping of docid to answer text
for _,row in test_df.iterrows():
    if row["qid"] not in question_map:
        question_map[row["qid"]] = row["question"]
    if row["answer"] not in label_map:
        label_map[row["answer"]] = row["docid"]

# Original Query Expansion

In [10]:
#Performs basic data cleaning on query: standardize capitalization to lower, remove punctuations, remove redundant whitespaces
def basic_cleaning(query):
    query = str(query)
    query = query.lower()
    query = re.sub(r'[^\w\s]','',query)
    query = ' '.join(query.split())
    return query

# Extract nouns
def nouns_only(query):
    try:
        tagged_text = nltk.tag.pos_tag(query.split())
        nouns_list = [word for word,tag in tagged_text if  tag == 'NNP' or tag == 'NNPS' or tag=="NN" or tag=="NNS"]
        return list(set(nouns_list))
    except:
        return []
    
def query_noun_mapping(query_nouns):
    synonym_dict = {}
    for query_noun in query_nouns:
        try:
            closest_noun = wn.synsets(query_noun)[0].lemmas()[1].name()
            synonym_dict[query_noun] = closest_noun
        except:
            pass
    return synonym_dict

def query_expansion(query):
    try:
        clean_query = basic_cleaning(query)
        query_nouns_list = nouns_only(clean_query)
        if len(query_nouns_list)==0:
            return query
        else:
            synonym_dict = query_noun_mapping(query_nouns_list)
        if len(synonym_dict.keys()) == 0:
            return query
        else:
            for k,v in synonym_dict.items():
                new_string = f"{k} and {v}"
                query = query.replace(k,new_string)
        return query
    except:
        return query

In [14]:
test_answer_list = test_df["answer"].tolist()
predictions = []
count = 1
for k,v in question_map.items():
    if count%100==0:
        print(count)
    try:
        v = query_expansion(v)
    except:
        pass
    question_embedding = bi_encoder_model.encode(v)
    answer_similiarity = {}
    for i,embed in enumerate(test_answer_embeddings):
        answer_similiarity[i] = np.dot(question_embedding,embed)
    answer_similiarity = {k: v for k, v in sorted(answer_similiarity.items(), key=lambda item: item[1], reverse=True)}
    top_20_hits = []
    for item in list(answer_similiarity)[:20]:
        top_20_hits.append(test_answer_list[item])
    cross_encoder_answer_similiarity = {}
    for hit in top_20_hits:
        cross_encoder_answer_similiarity[hit] = cross_encoder_model.predict([v,hit])
    cross_encoder_answer_similiarity = {k: v for k, v in sorted(cross_encoder_answer_similiarity.items(), key=lambda item: item[1], reverse=True)}
    label_index = label_map[list(cross_encoder_answer_similiarity.keys())[0]]
    predictions.append([label_index])
    count+=1  

In [ ]:
#save prediction and results
results = {"labels":labels,"predictions":predictions}
with open("../7.Evaluate/query_expansion.pkl", 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)

# New Domain-Based Query Expansion

In [13]:
# Load an spacy model (supported models are "es", "en" and "pt") 
nlp = spacy.load('en_core_web_sm')
# Spacy 3.x
nlp.add_pipe("spacy_wordnet", after='tagger', config={'lang': nlp.lang})
# Spacy 2.x
# self.nlp_en.add_pipe(WordnetAnnotator(self.nlp_en.lang))
token = nlp('prices')[0]

# wordnet object link spacy token with nltk wordnet interface by giving acces to
# synsets and lemmas 
token._.wordnet.synsets()
token._.wordnet.lemmas()

# And automatically tags with wordnet domains
token._.wordnet.wordnet_domains()

# query expansion hyperparameters
number_of_domain_lemma = 2
domains = ['insurance', 'banking', 'finance', 'money']
detected_pos = ['NOUN']

def basic_cleaning(query):
    #Performs basic data cleaning on query: standardize capitalization to lower, remove punctuations, remove redundant whitespaces
    query = str(query)
    query = query.lower()
    query = re.sub(r'[^\w\s]','',query)
    query = ' '.join(query.split())
    return query

def get_similar_words(current_token, synonyms_list, number_of_domain_lemma=number_of_domain_lemma):
    result = []
    if number_of_domain_lemma > len(synonyms_list):
        return synonyms_list
    for i in range(number_of_domain_lemma):
        synonym = synonyms_list[i]
        result.append(synonym)

    if current_token not in result:
        result[number_of_domain_lemma-1] = current_token

    return result

def query_expansion_v2(query, domains=domains, number_of_domain_lemma=number_of_domain_lemma, detected_pos=detected_pos):
    """
    @params:
        query: the question to be query-expanded
        domains: the specific domains to get the synonyms from, eg. some possible_domains are ['finance', 'banking', 'insurance', 'money', 'economy']
        number_of_domain_lemma: the number of synonyms you want, including the original word
    """
    enriched_sentence = []
    sentence = nlp(query)
    # For each token in the sentence
    for token in sentence:
        # print(token.text, token.pos_)
        # We get those synsets within the desired domains
        synsets = token._.wordnet.wordnet_synsets_for_domain(domains)

        if not synsets: 
            enriched_sentence.append(token.text)
        else:
            # If we found a synset in the economy domains and the word is a noun
            # we get the variants and add them to the enriched sentence
            if token.pos_ in detected_pos:
                # lemmas_for_synset = [lemma for s in synsets for lemma in s.lemma_names()] # 2d list comprehension
                lemmas_for_synset = []
                for s in synsets:
                    # print(s.lemma_names())
                    for lemma in s.lemma_names():
                        # print(lemma)
                        lemmas_for_synset.append(lemma)

                synonyms = get_similar_words(token.text, lemmas_for_synset, number_of_domain_lemma)
                synonyms = list(dict.fromkeys(synonyms))
                enriched_sentence.append('{}'.format(' '.join(synonyms)))
            else:
                enriched_sentence.append(token.text)

    enriched_sentence = " ".join(enriched_sentence)
    enriched_sentence = basic_cleaning(enriched_sentence)
    enriched_sentence = enriched_sentence.replace("_"," ")
    return enriched_sentence

In [14]:
# testing query expansion function
for i in range(10):
    test_query = list(question_map.values())[i]
    #test_query = question_map[7817]
    test_domain=['insurance', 'banking', 'finance', 'money']
    print("Original:", test_query)
    print("Query Expansion Result:", query_expansion_v2(test_query))
    print()

Original: Can you have a positive return with a balance below cost basis?
Query Expansion Result: can you have a positive tax return return with a balance below cost monetary value basis

Original: Oversimplify it for me: the correct order of investing
Query Expansion Result: oversimplify it for me the correct order purchase order of investing

Original: Definition of “U.S. source” for US non-resident alien capital gains tax
Query Expansion Result: definition of us beginning source for us non resident alien capital working capital gains tax taxation

Original: Are car buying services worth it?
Query Expansion Result: are car auto buy buying services service worth it

Original: What makes a Company's Stock prices go up or down?
Query Expansion Result: what makes a company company s stock monetary value prices go up or down

Original: Tax deductions on empty property
Query Expansion Result: tax tax tax writeoff deductions on empty property belongings

Original: I am the sole owner of an 

In [16]:
# Get predictions with new query expansion code

# query expansion hyperparameters
number_of_domain_lemma = 2
domains = ['insurance', 'banking', 'finance', 'money']
detected_pos = ['NOUN',"PROPN"]

test_answer_list = test_df["answer"].tolist()
updated_predictions = []
count = 1
for k,v in question_map.items():
    if count%100==0:
        print(count)
    try:
        v = query_expansion_v2(v, domains, number_of_domain_lemma, detected_pos)
    except:
        pass
    question_embedding = bi_encoder_model.encode(v)
    answer_similiarity = {}
    for i,embed in enumerate(test_answer_embeddings):
        answer_similiarity[i] = np.dot(question_embedding,embed)
    answer_similiarity = {k: v for k, v in sorted(answer_similiarity.items(), key=lambda item: item[1], reverse=True)}
    top_20_hits = []
    for item in list(answer_similiarity)[:20]:
        top_20_hits.append(test_answer_list[item])
    cross_encoder_answer_similiarity = {}
    for hit in top_20_hits:
        cross_encoder_answer_similiarity[hit] = cross_encoder_model.predict([v,hit])
    cross_encoder_answer_similiarity = {k: v for k, v in sorted(cross_encoder_answer_similiarity.items(), key=lambda item: item[1], reverse=True)}
    label_index = label_map[list(cross_encoder_answer_similiarity.keys())[0]]
    updated_predictions.append([label_index])
    count+=1  

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500


In [18]:
#save prediction and results
results = {"labels":labels,"predictions":updated_predictions}
with open("../7.Evaluate/updated_query_expansion.pkl", 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)